In [17]:
import pandas as pd
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import sys
import os
sys.path.insert(0,'..')
from src.etl.etl import *
from src.scripts.scrape import *
import random
import time
import extcolors
import urllib.request
import urllib

# Data paparation
1. Get all the company logo's img url (ending with .png)
2. download all the img store it in the data logo img folder
3. every time download an image, get the rgb of the img
4. store the rgb in another dataframe


In [57]:
data_path= os.path.join(os.path.abspath(os.pardir),"data\\cleaned_websites.txt")
ranking = pd.read_csv(data_path,delimiter="\t")

In [121]:
first_200 = ranking.loc[0:199]
first_200["imgUrl"] = None

C:\Users\Wendy W\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [103]:
#url_list = [0]*200

In [115]:
class_name_for_logos = "summary__company-main-logo"
for i in range(1,200):
    
    url = first_200.loc[i]["craftWebsite"]
    d = initialize_driver(url)
    d.get(url)
    time.sleep(1)
    try:
        logo_url = d.find_elements_by_class_name(class_name_for_logos)[0].get_attribute("src")
    except:
        print("Couldn't find logo for this company")
        logo_url = None
    kill_driver(d)
    pause_second = random.randint(1,5)
    time.sleep(pause_second)
    url_list[i] = logo_url

Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company
Couldn't find logo for this company


In [122]:
first_200["imgUrl"] = url_list

C:\Users\Wendy W\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
data_path_img_url= os.path.join(os.path.abspath(os.pardir),"data/img_urls.txt")
img_urls = pd.read_csv(data_path_img_url)
img_urls.head()

,Unnamed: 0,companyName,Ranking Actual,craftWebsite,imgUrl
0,0,Walmart,1,https://craft.co/walmart,https://uploads3.craft.co/uploads/company/logo...
1,1,Amazon.com,2,https://craft.co/amazon,https://uploads3.craft.co/uploads/company/logo...
2,2,Exxon Mobil,3,https://craft.co/vsn-mobil,https://uploads4.craft.co/uploads/company/logo...
3,3,Apple,4,https://craft.co/apple,https://uploads4.craft.co/uploads/company/logo...
4,4,CVS Health,5,https://craft.co/cvs-health,https://uploads2.craft.co/uploads/company/logo...


## Download Images


In [11]:
imgFolder = os.path.join(os.path.abspath(os.pardir),"data/logos/")

In [19]:
def download_image_from_url(url,folder_location,name):
    '''
    url: the destination url of the image
    name: filename of the image to be stored, before the dot jpg
    folder_location: local file path of the image to be stored. e.g. data/test_data/
    '''
    extension = url.split(".")[-1]
    filename = folder_location+name+"."+extension
    
    urllib.request.urlretrieve(url, filename)
    return filename

In [31]:
colors_list = [0] * 200
for i in range(0,len(img_urls)):
    
    imgUrl = img_urls.loc[i]["imgUrl"]
    if not isinstance(imgUrl,str):
        print(imgUrl)
        continue
    fileName = str(i).zfill(3)
    filename = download_image_from_url(imgUrl,imgFolder,fileName)
    colors, pixel_count = extcolors.extract_from_path(filename)
    colors_list[i] = colors

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [34]:
img_urls["color_rgb"] = colors_list

In [40]:
img_urls["numberOfColor"] = img_urls["color_rgb"].apply(lambda x:len(x) if (isinstance(x,list)) else 0)

In [43]:
rgb_file = os.path.join(os.path.abspath(os.pardir),"data/rgb.csv")
img_urls.drop("Unnamed: 0",axis=1).to_csv(rgb_file)

In [44]:
json_path = os.path.join(os.path.abspath(os.pardir),"config/credentials/credentials.json")
write_to_worksheet(rgb_file,json_path,"RGB of 0_200 Logos companies Fortune 500",["caiwei.wang.25@gmail.com"])

Created new spreadsheet:RGB of 0_200 Logos companies Fortune 500 From:/home/caw062/balance_scrape/data/rgb.csv
Sent to caiwei.wang.25@gmail.com
